In [ ]:
# Load mssparkutils
import notebookutils

# Get the current workspace ID
workspace_id = notebookutils.runtime.context.get("currentWorkspaceId")
# print(f'Workspace ID: {workspace_id}')


In [ ]:
######################################################################################### 
# Read secretes from Azure Key Vault
#########################################################################################
## This is the name of my Azure Key Vault 
key_vault = "https://domain.vault.azure.net/"
## I have stored my tenant id as one of the secrets to make it easier to use when needed 
tenant = mssparkutils.credentials.getSecret(key_vault , "tenantid") 
## This is my application Id for my service principal account 
client = mssparkutils.credentials.getSecret(key_vault , "pbi-sp-applicationid") 
## This is my Client Secret for my service principal account 
client_secret = mssparkutils.credentials.getSecret(key_vault , "powerbi-sp-clientsecret")  

######################################################################################### 
# Authentication - Replace string variables with your relevant values 
#########################################################################################  

import json, requests, pandas as pd 
import datetime  

try: 
    from azure.identity import ClientSecretCredential 
except Exception:
     !pip install azure.identity 
     from azure.identity import ClientSecretCredential 

# Generates the access token for the Service Principal 
api = 'https://analysis.windows.net/powerbi/api/.default' 
auth = ClientSecretCredential(authority = 'https://login.microsoftonline.com/', 
               tenant_id = tenant, 
               client_id = client, 
               client_secret = client_secret) 
access_token = auth.get_token(api)
access_token = access_token.token 

## This is where I store my header with the Access Token, because this is required when authenticating 
## to the Power BI Admin APIs 
header = {'Authorization': f'Bearer {access_token}'}  

print('\nSuccessfully authenticated.')

In [ ]:
# Create Case Insensitive Warehouse
import pandas as pd
import requests

# API to Create Warehouse
api_url = f"https://api.fabric.microsoft.com/v1/workspaces/{workspace_id}/items"

display(api_url)

# JSON Payload to Post to create the Case Insensitive Collation
payload = { 
  "type": "warehouse", 
  "displayName": "WH_Service_Principal", 
  "description": "New Lakehouse Service Principal case-insensitive collation", 
  "creationPayload": { 
    "defaultCollation": "Latin1_General_100_CI_AS_KS_WS_SC_UTF8" 
  } 
}

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Checking the Payload Looks Good
display(payload)

# Post the API Response to create the Warehouse
response = requests.post(api_url, headers=headers, json=payload)
